works with kernel Python \[conda env:pangu] as constructed in README

run fengwu with 1 GPU and 20 GB memory (must be v100, not gp100 because gp100 only has 16 GB VRAM, but v100 has 32GB)

Make these the default casper modules `module save`

```
Currently Loaded Modules:
  1) ncarenv/24.12  (S)   3) ncarcompilers/1.0.0   5) ucx/1.17.0      7) hdf5/1.12.3    9) cudnn/9.2.0.82-12
  2) intel/2024.2.1       4) cuda/12.3.2           6) openmpi/5.0.6   8) netcdf/4.9.2  10) conda/latest
```

Inferences in conda env:ainwp is different by 0.0001 K from pangu env (after 240 hours)
conda env:ainwp is supposed to replicate realtime runs, but still 0.1 K different from realtime runs

In [ ]:
import argparse
import concurrent.futures
import os
import subprocess
from pathlib import Path
from typing import List, Union

import earthkit.data as ekd
import pandas as pd
import uxarray
import xarray
from hwt_mpas import MPASDataSource
from run_pangu import plot_ensemble
from run_pangu.s3_run_fengwu_ecmwf import fengwu_channels, pressure_levels, variables
from run_pangu.s3_run_pangu_ecmwf import run_inference, setup_model_sessions
from run_pangu.utils.xtime import xtime

ai_models_dir = Path("/glade/derecho/scratch/ahijevyc/ai-models")
date = pd.to_datetime("2024042500", format="%Y%m%d%H")
ic = "mpas"
fhr_end = 240

In [ ]:
!uname -a

In [ ]:
mpas_datasrc = MPASDataSource(0)
ds_mpas = mpas_datasrc(date)

ds_mpas

In [ ]:
ort_session_24, ort_session_6 = setup_model_sessions(ai_models_dir)

In [ ]:
output_dir = Path("/glade/derecho/scratch/ahijevyc/ai-models/output/panguweather") / date.strftime(
    "%Y%m%d%H"
)
for member in range(1, 11):
    all_exist = True
    for fhr in range(6, fhr_end+1, 6):
        output_filename = os.path.join(output_dir, f"pangu_{ic}{member}_pred_{fhr:03.0f}.nc")
        if not os.path.exists(output_filename):
            print(output_filename, 'no exists')
            all_exist = False
            break
    if all_exist:
        print(f"all {date} {ic}{member} exist")
        continue
    inferences = run_inference(ds_mpas.sel(member=member), ort_session_24, ort_session_6, fhr_end)
    for fcst in inferences:
        fhr = fcst.prediction_timedelta.squeeze() / pd.to_timedelta("1h")
        output_filename = os.path.join(output_dir, f"pangu_{ic}{member}_pred_{fhr:03.0f}.nc")
        print(output_filename)
        fcst.to_netcdf(output_filename)

In [ ]:
inferences = xarray.open_mfdataset(output_dir.glob(f"pangu_{ic}{member}_pred_???.nc"))
inferences.sel(channel="z1000").squeeze().__xarray_dataarray_variable__.plot(col="prediction_timedelta", col_wrap=6)

In [ ]:
inferences